# Section 03 — Custom CNN Model

Here we implement a custom convolutional neural network trained from scratch.

The model contains:
- Three convolution blocks (Conv → ReLU → MaxPool)
- A fully connected layer producing a 512-dimensional vector
- A final classification layer with four output units

The training loop uses:
- Adam optimizer
- CrossEntropy loss
- Best-model checkpointing based on validation accuracy

This model serves as the baseline for comparing against pretrained architectures.

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMAGE_SIZE = 224
BATCH_SIZE = 32

train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
val_dataset   = datasets.ImageFolder("data/validation",   transform=test_val_transforms)
test_dataset  = datasets.ImageFolder("data/test",  transform=test_val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)

print("✔ Dataloaders ready!")

✔ Dataloaders ready!


In [7]:
# import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


def train_cnn(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_acc = 0.0   # track best accuracy

    for epoch in range(num_epochs):

        # Training phase
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()
        correct = 0
        total = 0

        # Disable gradient calculation for validation
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                # Calculate accuracy
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total

        print(f"Epoch {epoch+1}/{num_epochs} "
              f"| Train Loss: {avg_train_loss:.4f} "
              f"| Val Acc: {val_acc:.4f}")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "cnn_model.pth")
            print(f"Best model updated! Saved with Val Acc = {best_val_acc:.4f}")

    print("Training completed.")
    print(f"Best Validation Accuracy: {best_val_acc:.4f}")

    return model

In [10]:
cnn_model = CNNModel(num_classes=4)

model_trained = train_cnn(
    model=cnn_model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    learning_rate=0.001
)

print("CNN training complete!")

Epoch 1/20 | Train Loss: 0.7817 | Val Acc: 0.7493
Best model updated! Saved with Val Acc = 0.7493
Epoch 2/20 | Train Loss: 0.5941 | Val Acc: 0.8044
Best model updated! Saved with Val Acc = 0.8044
Epoch 3/20 | Train Loss: 0.4901 | Val Acc: 0.8239
Best model updated! Saved with Val Acc = 0.8239
Epoch 4/20 | Train Loss: 0.4276 | Val Acc: 0.8466
Best model updated! Saved with Val Acc = 0.8466
Epoch 5/20 | Train Loss: 0.3878 | Val Acc: 0.8403
Epoch 6/20 | Train Loss: 0.3444 | Val Acc: 0.8617
Best model updated! Saved with Val Acc = 0.8617
Epoch 7/20 | Train Loss: 0.3163 | Val Acc: 0.8557
Epoch 8/20 | Train Loss: 0.2923 | Val Acc: 0.8652
Best model updated! Saved with Val Acc = 0.8652
Epoch 9/20 | Train Loss: 0.2831 | Val Acc: 0.8724
Best model updated! Saved with Val Acc = 0.8724
Epoch 10/20 | Train Loss: 0.2586 | Val Acc: 0.8731
Best model updated! Saved with Val Acc = 0.8731
Epoch 11/20 | Train Loss: 0.2512 | Val Acc: 0.8557
Epoch 12/20 | Train Loss: 0.2289 | Val Acc: 0.8885
Best model up